In [1]:
import os
import pandas as pd
from datetime import datetime

# import tensorflow as tf
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator


def extract_scalars_from_event_file(event_file):
    event_acc = EventAccumulator(event_file)
    event_acc.Reload()
    

    tags = event_acc.Tags()["scalars"]

    all_scalars = {}
    for tag in tags:
        events = event_acc.Scalars(tag)
        values = [event.value for event in events]
        steps = [event.step for event in events]

        timestamps = [pd.to_datetime(event.wall_time, unit='s') for event in events]
        all_scalars[tag] = pd.DataFrame({"step": steps, "value": values, "timestamp": timestamps})

    return all_scalars


files_t = []
def event_files_to_dataframe(directory):
    all_scalars_dfs = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.startswith("events.out.tfevents"):
                event_file = os.path.join(root, file)

                file_from = pd.to_datetime(file.split(".")[3], unit='s')
                files_t.append(file_from)

                # print(file_from)
                if file_from < date_obj:
                    continue

                scalars = extract_scalars_from_event_file(event_file)
                for tag, df in scalars.items():
                    df["tag"] = tag
                    all_scalars_dfs.append(df)

    all_scalars_df = pd.concat(all_scalars_dfs, ignore_index=True)
    return all_scalars_df

df_all = []
for i in range(5):
    # Directory containing your event files
    log_dir = f"./dfl_5_fully_connected/participant_{i}"
    date_str = "02/07/2024 11:05:30" # minus two hours for timezone

    date_obj = datetime.strptime(date_str, "%d/%m/%Y %H:%M:%S")

    df = event_files_to_dataframe(log_dir)
    print(df.head())

    df.sort_values(by="timestamp", inplace=True)

    files_t.sort(reverse=True)
    print(files_t[0], files_t[-1])

    df[df["tag"]=='Test/Loss']["timestamp"].max()

    test_df = df[df["tag"].str.startswith("Test/")].groupby("tag").last().copy()
    print(test_df)
    test_df["participant"] = f"participant_{i}"
    df_all.append(test_df)

   step     value                     timestamp             tag
0  5046  8.000000 2024-07-02 11:10:39.705205248           Round
1  2523  0.055050 2024-07-02 11:10:40.563845632       Test/Loss
2  2523  0.946850 2024-07-02 11:10:40.563906048   Test/Accuracy
3  2523  0.950248 2024-07-02 11:10:40.563930624  Test/Precision
4  2523  0.946850 2024-07-02 11:10:40.563952896     Test/Recall
2024-07-02 11:11:22 2024-07-02 11:07:03
                step     value                     timestamp
tag                                                         
Test/Accuracy   3175  0.952468 2024-07-02 11:11:20.789978880
Test/F1Score    3175  0.952639 2024-07-02 11:11:20.790064128
Test/Loss       3175  0.036808 2024-07-02 11:11:20.789894656
Test/Precision  3175  0.956332 2024-07-02 11:11:20.790010624
Test/Recall     3175  0.952468 2024-07-02 11:11:20.790038784
   step  value                     timestamp                    tag
0     0    0.0 2024-07-02 11:06:06.678570496                  Round
1    12   29.

In [2]:
concat_df = pd.concat(df_all)
concat_df

,step,value,timestamp,participant
tag,,,,
Test/Accuracy,3175,0.952468,2024-07-02 11:11:20.789978880,participant_0
Test/F1Score,3175,0.952639,2024-07-02 11:11:20.790064128,participant_0
Test/Loss,3175,0.036808,2024-07-02 11:11:20.789894656,participant_0
Test/Precision,3175,0.956332,2024-07-02 11:11:20.790010624,participant_0
Test/Recall,3175,0.952468,2024-07-02 11:11:20.790038784,participant_0
Test/Accuracy,3055,0.958019,2024-07-02 11:11:21.096433664,participant_1
Test/F1Score,3055,0.955860,2024-07-02 11:11:21.096509440,participant_1
Test/Loss,3055,0.040076,2024-07-02 11:11:21.096384512,participant_1
Test/Precision,3055,0.957277,2024-07-02 11:11:21.096461568,participant_1


In [3]:
concat_df.groupby("tag").mean()

/tmp/ipykernel_39949/1972488931.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  concat_df.groupby("tag").mean()


,step,value
tag,,
Test/Accuracy,3126.8,0.954397
Test/F1Score,3126.8,0.953236
Test/Loss,3126.8,0.038096
Test/Precision,3126.8,0.956550
Test/Recall,3126.8,0.954397


In [4]:
concat_df.to_csv("results_csvs/dfl_5_fully_connected.csv")